# 第5章: 係り受け解析
---
夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [94]:
!head -25 neko.txt.cabocha

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
* 2 -1D 0/0 0.000000
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。
EOS
EOS



## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラス`Morph`を実装せよ．このクラスは表層形（`surface`），基本形（`base`），品詞（`pos`），品詞細分類1（`pos1`）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文を`Morph`オブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [35]:
%%file q40.py
import argparse
from itertools import groupby
from string import printable
import sys


class Morph:
    """cabocha lattice formatファイルの1行を読み込む"""
    __slots__ = ('surface', 'pos', 'pos1', 'base')
    exceptions = frozenset(printable)
    # 吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
    def __init__(self, line):
        self.surface, temp = line.rstrip().split('\t')
        inf = temp.split(',')
        self.pos = inf[0]
        self.pos1 = inf[1]
        # 4章同様, 半角文字の基本形問題
        if self.surface in self.exceptions:
            self.base = self.surface
        else:
            self.base = inf[6]
    @classmethod
    def load_cabocha(cls, fi):
        """cabocha lattice formatファイルからMorphインスタンスを生成"""
        for is_eos, sentence in groupby(fi, key=lambda x: x == 'EOS\n'):
            if not is_eos:
                yield [cls(line) for line in sentence if not line.startswith('* ')]
                # startswith('*')だと表層形が「*」のときにまずい
    
    def __str__(self):
        return self.surface
    
    def __repr__(self):
        return 'q40.Morph({})'.format(', '.join((self.surface, self.pos, self.pos1, self.base)))
    

def main():
    sent_idx = arg_int()
    for i, sent_lis in enumerate(Morph.load_cabocha(sys.stdin), start=1):
        if i == sent_idx:
            print(*sent_lis)
            print(repr(sent_lis))
            break

def arg_int():
    parser = argparse.ArgumentParser()
    parser.add_argument('-n', '--number', default='1', type=int)
    args = parser.parse_args()
    return args.number


if __name__ == '__main__':
    main()


Overwriting q40.py


In [57]:
!python q40.py -n3 < neko.txt.cabocha

名前 は まだ 無い 。
[q40.Morph(名前, 名詞, 一般, 名前), q40.Morph(は, 助詞, 係助詞, は), q40.Morph(まだ, 副詞, 助詞類接続, まだ), q40.Morph(無い, 形容詞, 自立, 無い), q40.Morph(。, 記号, 句点, 。)]


In [89]:
import q40
q40.Morph('"	名詞,サ変接続,*,*,*,*,*')

q40.Morph(", 名詞, サ変接続, ")

In [90]:
q40.Morph(',	記号,読点,*,*,*,*,",",",",",",,')

q40.Morph(,, 記号, 読点, ,)

## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [36]:
%%file q41.py
from collections import defaultdict
from itertools import groupby
import sys

from q40 import Morph, arg_int


class Chunk:
    """cabocha lattice formatファイルから文節を読み込む"""
    __slots__ = ('idx', 'dst', 'morphs', 'srcs')
    
    # * 0 2D 0/0 -0.764522
    def __init__(self, line):
        info = line.rstrip().split()
        self.idx = int(info[1])
        self.dst = int(info[2].rstrip("D"))
        self.morphs = []
        self.srcs = []
    
    def __str__(self):
        return ''.join([morph.surface for morph in self.morphs])
    
    def __repr__(self):
        return 'q41.Chunk({}, {})'.format(self.idx, self.dst)
    
    def srcs_append(self, src_idx):
        """係り元文節インデックスを追加"""
        self.srcs.append(src_idx)
    
    def morphs_append(self, line):
        """形態素を追加"""
        self.morphs.append(Morph(line))
    
    def chunk2str(self):
        """記号を取り除いた文節の表層形を返す"""
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])
    
    def contain_pos(self, pos):
        """文節中にある品詞が存在するかどうかを返す"""
        return pos in (morph.pos for morph in self.morphs)

    
class Sentence:
    """cabocha lattice formatファイルから文を読み込む。Chunkクラスのヘルパー"""
    __slots__ = ('chunks', 'idx')
    
    def __init__(self, sent_lines):
        self.chunks = []
        ch_append = self.chunks.append
        for line in sent_lines:                    
            if line.startswith('* '):
                ch_append(Chunk(line))
            else:
                self.chunks[-1].morphs_append(line)
        # srcsをappendしたいがためのクラス
        for chunk in self.chunks:
            if chunk.dst != -1:
                self.chunks[chunk.dst].srcs_append(chunk.idx)
    
    def __str__(self):
        return ' '.join([morph.surface for chunk in self.chunks for morph in chunk.morphs])
    
    @classmethod
    def load_cabocha(cls, fi):
        """cabocha lattice formatファイルからSentenceインスタンスを生成"""
        for is_eos, sentence in groupby(fi, key=lambda x: x == 'EOS\n'):
            if not is_eos:
                yield cls(sentence)
    
    def print_dep_idx(self):
        """係り元文節インデックスと係り先文節インデックスを表示"""
        for chunk in self.chunks:
            print('{}:{} => {}'.format(chunk.idx, chunk, chunk.dst))
    
    def print_dep(self):
        """係り元文節と係り先文節の表層をタブ区切りで表示"""
        for chunk in self.chunks:
            if chunk.dst != -1:
                print('{}\t{}'.format(chunk.chunk2str(), self.chunks[chunk.dst].chunk2str()))
                
    def dep_edge(self):
        """pydotで係り受けを出力する用"""
        return [(chunk.chunk2str(), self.chunks[chunk.dst].chunk2str())
                    for chunk in self.chunks if chunk.dst != -1]
                
    def print_noun_verb_dep(self):
        """名詞を含む文節が動詞を含む文節に係るものを抽出"""
        for chunk in self.chunks:
            if chunk.contain_pos('名詞') and self.chunks[chunk.dst].contain_pos('動詞'):
                print('{}\t{}'.format(chunk.chunk2str(), self.chunks[chunk.dst].chunk2str()))
    
    
    def trace_dep_path(self):
        """名詞を含む文節からrootまでの係り受けパスを追跡"""
        path = []
        ph_append = path.append
        for chunk in self.chunks:
            if chunk.contain_pos('名詞'):
                ph_append(chunk)
                d = chunk.dst
                while d != -1:
                    ph_append(self.chunks[d])
                    d = self.chunks[d].dst
                
                yield path
                path.clear()
                    
                    
def main():
    sent_id = arg_int()
    for i, sent in enumerate(Sentence.load_cabocha(sys.stdin), start=1):
        if i == sent_id:
            sent.print_dep_idx()
            break


if __name__ == '__main__':
    main()


Overwriting q41.py


In [59]:
!python q41.py -n8 < neko.txt.cabocha

0:この => 1
1:書生というのは => 7
2:時々 => 4
3:我々を => 4
4:捕えて => 5
5:煮て => 6
6:食うという => 7
7:話である。 => -1


## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [60]:
%%file q42.py
import sys

from q40 import arg_int
from q41 import Sentence


def main():
    sent_id = arg_int()
    for i, sent in enumerate(Sentence.load_cabocha(sys.stdin), start=1):
        if i < sent_id:
            sent.print_dep()
        else:
            break

if __name__ == '__main__':
    main()


Overwriting q42.py


In [63]:
!python q42.py -n5 < neko.txt.cabocha

	猫である
吾輩は	猫である
名前は	無い
まだ	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [155]:
%%file q43.py
import sys

from q40 import arg_int
from q41 import Sentence


def main():
    sent_id = arg_int()
    for i, sent in enumerate(Sentence.load_cabocha(sys.stdin), start=1):
        if i < sent_id:
            sent.print_noun_verb_dep()
        else:
            break

if __name__ == '__main__':
    main()

Overwriting q43.py


In [65]:
!python q43.py -n10 < neko.txt.cabocha

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
記憶している	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて


# 44. 係り受け木の可視化

与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．


In [66]:
%%file q44.py
import sys

from q40 import arg_int
from q41 import Sentence
import pydot


def main():
    sent_id = arg_int()
    for i, sent in enumerate(Sentence.load_cabocha(sys.stdin), start=1):
        if i == sent_id:
            edges = sent.dep_edge()
            n = pydot.Node('node')
            n.fontsize = 9
            graph = pydot.graph_from_edges(edges, directed=True)
            graph.add_node(n)
            graph.write_jpeg("dep_tree_neko{}.jpg".format(i))
            break

if __name__ == "__main__":
    main()


Overwriting q44.py


In [67]:
!python q44.py  -n8 < neko.txt.cabocha

In [68]:
!open dep_tree_neko8.jpg

In [53]:
!scp -P 2022 dep_tree_neko8.jpg asano@cocoa.cl.ecei.tohoku.ac.jp:/home/asano/public_html/nlp100/

dep_tree_neko8.jpg                            100%   28KB  28.3KB/s   00:00    


![neko](http://www.cl.ecei.tohoku.ac.jp/~asano/nlp100/dep_tree_neko8.jpg)

# 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```始める  で
見る    は を```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語と格パターンの組み合わせ
- 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

## 注
- 格を調査したいなら、格助詞を抽出するべき
- 接続助詞「が」と格助詞「が」は別物
- 問題文中の例でも係助詞「は」が抽出されてるのは大変まずい気が
- 以下のコードは泣く泣く問題文に従うものとする

In [79]:
sorted(['かがく', 'かかし']) # 文字コードに基づく辞書式順序にしてくれる模様

['かかし', 'かがく']

In [106]:
%%file q45.py
import sys

from q41 import Sentence


def main():
    for sentence in Sentence.load_cabocha(sys.stdin):
        case_pattern(sentence)
        
def case_pattern(sent):
    for chunk in sent.chunks:
        for morph in chunk.morphs:
            if morph.pos == '動詞':
                verb = morph.base
                particles = [] # 助詞のリスト
                for src in chunk.srcs:
                    # 分節内で一番右の助詞を追加していく
                    particles.extend([word.base for word in sent.chunks[src].morphs 
                                         if word.pos == '助詞'][-1:])
                particles.sort()
                print('{}\t{}'.format(verb, ' '.join(particles)))
                # 一番左の動詞しか使わないのでさっさと抜ける
                break

            
if __name__ == "__main__":
    main()


Overwriting q45.py


In [107]:
!python q45.py < neko.txt.cabocha | sort | uniq -c | sort -rn | head -20

    704 云う	と
    452 する	を
    435 する	
    333 思う	と
    202 ある	が
    199 なる	に
    188 する	に
    175 見る	て
    159 する	と
    122 云う	
    117 する	が
    113 する	に を
    108 なる	
     98 見る	を
     97 見える	と
     94 ある	
     90 する	て を
     89 いる	
     85 する	は
     80 見る	
sort: write failed: 'standard output': Broken pipe
sort: write error


In [70]:
!python q45.py < neko.txt.cabocha | grep -E "^(する|見る|与える)\s" | sort | uniq -c | sort -nr | head -20

    452 する	を
    435 する	
    188 する	に
    175 見る	て
    159 する	と
    117 する	が
     98 見る	を
     88 する	て を
     85 する	は
     80 見る	
     61 する	を に
     61 する	て
     60 する	も
     54 する	が を
     52 する	に を
     51 する	から
     44 する	と を
     44 する	で を
     40 する	の
     37 する	は を
sort: fflush failed: 'standard output': Broken pipe
sort: write error


# 46. 動詞の格フレーム情報の抽出

45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．
- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```始める  で      ここで
見る    は を   吾輩は ものを```

In [89]:
sorted([('い', 5), ('あ', 8)])

[('あ', 8), ('い', 5)]

In [114]:
%%file q46.py
import sys

from q41 import Sentence


def main():
    for sentence in Sentence.load_cabocha(sys.stdin):
        pred_case_arg(sentence)
        
def pred_case_arg(sent):
    for chunk in sent.chunks:
        for morph in chunk.morphs:
            if morph.pos == '動詞':
                verb = morph.base
                particle_chunks = []
                for src in chunk.srcs:
                    # (助詞, 係り元の分節の表層)
                    particle_chunks.extend([(word.base, sent.chunks[src].chunk2str()) 
                                            for word in sent.chunks[src].morphs if word.pos == '助詞'][-1:])
                if particle_chunks:
                    particle_chunks.sort()
                    particles, chunks = zip(*particle_chunks)
                else:
                    particles, chunks = [], []
                
                print('{}\t{}\t{}'.format(verb, ' '.join(particles), ' '.join(chunks)))
                break

            
if __name__ == "__main__":
    main()


Overwriting q46.py


In [117]:
!python q46.py < neko.txt.cabocha | grep -E "^(する)\s" | sort | uniq -c | sort -nr | head -20

    435 する		
     28 する	を	顔を
     22 する	を	事を
     13 する	に	事に
      9 する	が	声が
      8 する	を	顔付を
      8 する	を	真似を
      7 する	を	人を
      7 する	が	気が
      6 する	を	運動を
      6 する	を	懐手を
      6 する	と	ものと
      5 する	を	知らん顔を
      5 する	に	くらいに
      5 する	を	話しを
      5 する	と	あると
      5 する	に	大に
      5 する	が	音が
      4 する	を	返事を
      4 する	を	喧嘩を
sort: write failed: 'standard output': Broken pipe
sort: write error


# 47. 機能動詞構文のマイニング

動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．

```返事をする      と に は      及ばんさと 手紙に 主人は```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
- コーパス中で頻出する述語（サ変接続名詞+を+動詞）
- コーパス中で頻出する述語と助詞パターン


In [2]:
%%file q47.py
import sys

from q41 import Sentence


def main():
    for sentence in Sentence.load_cabocha(sys.stdin):
        sahen_case_arg(sentence)
        
def sahen_case_arg(sent):
    # サ変名詞＋動詞を抽出するためのフラグ
    sahen_flag = 0
    for chunk in sent.chunks:
        for morph in chunk.morphs:
            if sahen_flag == 0 and morph.pos1 == 'サ変接続':
                sahen_flag = 1
                sahen = morph.surface
            elif sahen_flag == 1 and morph.base == 'を' and morph.pos == '助詞':
                sahen_flag = 2
            elif sahen_flag == 2 and morph.pos == '動詞':
                sahen_wo = sahen + 'を'
                verb = morph.base
                particle_chunks = []
                for src in chunk.srcs:
                    # (助詞, 係り元の分節の表層)
                    particle_chunks.extend([(word.base, sent.chunks[src].chunk2str()) for word in sent.chunks[src].morphs 
                                     if word.pos == '助詞'][-1:])
                for j, part_chunk in enumerate(particle_chunks[:]):
                    if sahen_wo in part_chunk[1]:
                        del particle_chunks[j]
                
                if particle_chunks:
                    particle_chunks.sort()
                    particles, chunks = zip(*particle_chunks)
                else:
                    particles, chunks = [], []

                print('{}\t{}\t{}'.format(sahen_wo + verb, ' '.join(particles), ' '.join(chunks)))
                sahen_flag = 0
                break
            else:
                sahen_flag = 0

            
if __name__ == "__main__":
    main()

Overwriting q47.py


In [39]:
!python q47.py < neko.txt.cabocha | sort | uniq -c | sort -nr | head

      8 真似をする		
      6 喧嘩をする		
      4 運動をする		
      4 返事をする		
      4 話を聞く		
      3 話をする		
      2 御無沙汰をする		
      2 休養を要する	は	吾輩はまた
      2 深入りをする		
      2 遠慮をする		
sort: fflush failed: 'standard output': Broken pipe
sort: write error


In [ ]:
cut -f 1 work/case-pattern-wo.txt | sort | uniq -c | sort -nr | head

# 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．
- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を```"->"```で連結する

「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た```


In [37]:
%%file q48.py
import sys

from q40 import arg_int
from q41 import Sentence


def main():
    sent_id = arg_int()
    for i, sent in enumerate(Sentence.load_cabocha(sys.stdin), start=1):
        if i == sent_id:
            for chunks in sent.trace_dep_path():
                print(' -> '.join([chunk.chunk2str() for chunk in chunks]))
            break

if __name__ == '__main__':
    main()

Overwriting q48.py


In [38]:
!python q48.py -n6 < neko.txt.cabocha 

吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た


# 49. 名詞間の係り受けパスの抽出

文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi
とj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

- 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
- 文節iとjに含まれる名詞句はそれぞれ，XとYに置換する

また，係り受けパスの形状は，以下の2通りが考えられる．

- 文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節j
のパスを表示
- 上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を"|"で連結して表示

例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y```

In [27]:
%%file q49.py
#途中
import sys
from itertools import combinations

from q40 import arg_int
from q41 import Sentence


def main():
    sent_id = arg_int()
    for i, sent in enumerate(Sentence.load_cabocha(sys.stdin), start=1):
        if i == sent_id:
            for chunks in sent.trace_dep_path():
                for c1, c2 in combinations(chunks, 2):
                    if c2.contain_pos('名詞'):
                        
            break

if __name__ == '__main__':
    main()

Overwriting q49.py


In [28]:
!python q49.py -n6 < neko.txt.cabocha 

人間という
ものを
人間という
ものを
ものを
ものを
